In [7]:
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from tqdm import tqdm
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import optimizers as opt
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import losses
import pandas as pd
from sklearn.metrics import precision_score,recall_score,f1_score, cohen_kappa_score

In [8]:
DATA_PATH="../Vowel_Data/Formant/" # relative path ../ means up once

In [9]:
seed=42

In [10]:
def get_formant(file_path):
    formant=pd.read_csv(file_path,
                delim_whitespace=True,
                header=None,
                names=["time","F1","F2","F3","F4","F5"],
                dtype=np.float64,
                na_values="--undefined--")
    return formant

def get_all_formants(path=DATA_PATH):
    labels=labels=sorted(os.listdir(path))
    all_formant=pd.DataFrame() #empty dataframe to store all_formants
    
    for label_index,label in enumerate(labels):
        folder_formant=pd.DataFrame() #empty dataframe to store formants of indivuals vowels/words i.e. folders

        formant_files=[path + label + '/' + file for file in sorted(os.listdir(path+'/'+label))]
        for file in tqdm(formant_files,"Reading Formants of label -'{}'".format(label)):
            #print(file)
            individual_formant=get_formant(file_path=file)
  
#             length=len(individual_formant)
#             F_i=individual_formant.loc[2:,'F1':'F5'].values
#             F_i1=individual_formant.loc[1:length-2,'F1':'F5'].values
#             F_i2=individual_formant.loc[:length-3,'F1':'F5'].values
#             assert len(F_i)==len(F_i1)==len(F_i2)
#             individual_formant.loc[2:,['F1_diff2','F2_diff2','F3_diff2','F4_diff2','F5_diff2']]=np.abs(F_i-2*F_i1+F_i2)
            
#             individual_formant=individual_formant.dropna()   #drop the first two rows as there are NaN values due to the difference
            
            folder_formant=folder_formant.append(individual_formant,ignore_index=True)
        
#         folder_formant['disp(F5-F1)']=folder_formant['F5']-folder_formant['F1']
#         folder_formant['disp(F4-F3)']=folder_formant['F4']-folder_formant['F3']
#         folder_formant['disp(F5-F3)']=folder_formant['F5']-folder_formant['F3']
#         folder_formant['disp(F5-F4)']=folder_formant['F5']-folder_formant['F4']
        
        folder_formant['label']=label_index  #new columns for encoding label
        
        all_formant=all_formant.append(folder_formant,ignore_index=True)     
    return all_formant

def get_train_test():
    all_formants=get_all_formants()
    
    y=all_formants['label'].values
 
    only_formants=all_formants.drop(labels=['time','label'],axis=1)
    
    standard_formants=(only_formants-only_formants.mean())/only_formants.std()
    
    X=standard_formants.values   

    assert X.shape[0] == len(y)
    return X,y

In [11]:
vowel_formant_features=get_all_formants()
# vowel_formant_features.to_excel("/home/rakibul/WORK/RESEARCH/Speech_Recognition/MAIN STUDY/GitHub Repo/Result/New_without_CNN/vowel_all_formant_features.xlsx")

Reading Formants of label -'Formant1':   0%|          | 0/40 [00:00<?, ?it/s]/tmp/ipykernel_22629/3260905446.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  folder_formant=folder_formant.append(individual_formant,ignore_index=True)
Reading Formants of label -'Formant1':   2%|▎         | 1/40 [00:00<00:14,  2.65it/s]/tmp/ipykernel_22629/3260905446.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  folder_formant=folder_formant.append(individual_formant,ignore_index=True)
/tmp/ipykernel_22629/3260905446.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  folder_formant=folder_formant.append(individual_formant,ignore_index=True)
/tmp/ipykernel_22629/3260905446.py:31: FutureWarning: The frame.append method is deprecated

In [12]:
vowel_formant_features=vowel_formant_features.drop(labels=['F4','F5'],axis=1) #reporting result for F1 to F3 only
vowel_formant_features

,time,F1,F2,F3,label
0,0.025,705.97,1456.68,2400.93,0
1,0.031,698.49,1496.51,2366.53,0
2,0.038,670.76,1514.62,2428.13,0
3,0.044,666.49,1524.23,2439.99,0
4,0.050,667.40,1524.64,2409.82,0
...,...,...,...,...,...
14156,0.331,314.53,2646.94,3283.76,6
14157,0.338,300.73,2592.58,3240.90,6
14158,0.344,291.53,2523.56,3304.41,6
14159,0.350,293.56,2454.07,3125.72,6


In [13]:
X,y=get_train_test()

Reading Formants of label -'Formant1':   0%|          | 0/40 [00:00<?, ?it/s]/tmp/ipykernel_22629/3260905446.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  folder_formant=folder_formant.append(individual_formant,ignore_index=True)
/tmp/ipykernel_22629/3260905446.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  folder_formant=folder_formant.append(individual_formant,ignore_index=True)
Reading Formants of label -'Formant1':   5%|▌         | 2/40 [00:00<00:02, 13.82it/s]/tmp/ipykernel_22629/3260905446.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  folder_formant=folder_formant.append(individual_formant,ignore_index=True)
/tmp/ipykernel_22629/3260905446.py:31: FutureWarning: The frame.append method is deprecated

In [14]:
# checking batch size

kfold=StratifiedKFold(n_splits=4,shuffle=True,random_state=seed)

for train,test in kfold.split(X,y):    
    print('batch size = '+str(X[train].shape[0]))

batch size = 10620
batch size = 10621
batch size = 10621
batch size = 10621


In [9]:
kfold=StratifiedKFold(n_splits=4,shuffle=True,random_state=seed)
cv_acc=[]
cv_AUC=[]
cv_f1=[]
cv_kappa=[]

for train,test in kfold.split(X,y):
    y_hot=to_categorical(y)
    
    model_v=Sequential()
    model_v.add(Dense(128, activation='tanh',input_shape=(X[train].shape[1],)))
    model_v.add(Dense(128, activation='tanh'))
    model_v.add(Dense(64, activation='tanh'))
    model_v.add(Dense(32, activation='tanh'))
    model_v.add(Dense(16, activation='tanh'))
    model_v.add(Dense(7, activation='softmax'))
    
    model_v.compile(
        optimizer=opt.Adam(learning_rate=0.005),
        loss='categorical_crossentropy',
        metrics=[['accuracy','AUC']]
    )
    model_v.fit(X[train], y_hot[train], epochs=300, batch_size=X[train].shape[0],verbose=0)
    
    print('batch size = '+str(X[train].shape[0]))  # checking batch size
    
    scores=model_v.evaluate(X[test],y_hot[test],verbose=0)
    
    print("%s: %.2f" % (model_v.metrics_names[1],scores[1]))
    print("%s: %.2f" % (model_v.metrics_names[2],scores[2]))
    
    #yhat_classes = model_v.predict_classes(X[test], verbose=0) #deprecated
    yhat_classes=np.argmax(model_v.predict(X[test]), axis=-1)
    
    f1 = f1_score(y[test], yhat_classes,average='weighted')
    print('F1: %.2f' % f1)
    
    kappa = cohen_kappa_score(y[test], yhat_classes)
    print('Cohens kappa: %.2f' % kappa)
    
    cv_acc.append(scores[1])
    cv_AUC.append(scores[2])
    cv_f1.append(f1)
    cv_kappa.append(kappa)
    
    print(' ')
    
print("Overall acc: %.2f (+/- %.2f)" % (np.mean(cv_acc),np.std(cv_acc)))
print("Overall AUC: %.2f (+/- %.2f)" % (np.mean(cv_AUC),np.std(cv_AUC)))
print("Overall f1: %.2f (+/- %.2f)" % (np.mean(cv_f1),np.std(cv_f1)))
print("Overall kappa: %.2f (+/- %.2f)" % (np.mean(cv_kappa),np.std(cv_kappa)))

In [ ]:
save_specifier='absolute_trans_3F_128_128_64_32_16_tanh_adam_005_batchfull_seed42'

In [ ]:
cv_data_df=pd.DataFrame(data={'acc':cv_acc,
                              'AUC':cv_AUC,
                              'F1':cv_f1,
                              'Kappa':cv_kappa}
                       )
cv_data_df.to_csv('/home/rakibul/WORK/RESEARCH/Speech_Recognition/MAIN STUDY/GitHub Repo/Result/New_without_CNN/vowel_cv_data_df_'+save_specifier+'.csv')

In [15]:
DATA_PATH="../Word_Data/Formant/" # relative path

In [16]:
def get_formant(file_path):
    formant=pd.read_csv(file_path,
                delim_whitespace=True,
                header=None,
                names=["time","F1","F2","F3","F4","F5"],
                dtype=np.float64,
                na_values="--undefined--")
    return formant

def get_all_formants(path=DATA_PATH):
    labels=labels=sorted(os.listdir(path))
    all_formant=pd.DataFrame() #empty dataframe to store all_formants
    
    for label_index,label in enumerate(labels):
        folder_formant=pd.DataFrame() #empty dataframe to store formants of indivuals vowels/words i.e. folders

        formant_files=[path + label + '/' + file for file in sorted(os.listdir(path+'/'+label))]
        for file in tqdm(formant_files,"Reading Formants of label -'{}'".format(label)):
            #print(file)
            individual_formant=get_formant(file_path=file)
  
#             length=len(individual_formant)
#             F_i=individual_formant.loc[2:,'F1':'F5'].values
#             F_i1=individual_formant.loc[1:length-2,'F1':'F5'].values
#             F_i2=individual_formant.loc[:length-3,'F1':'F5'].values
#             assert len(F_i)==len(F_i1)==len(F_i2)
            #individual_formant.loc[2:,['F1_diff2','F2_diff2','F3_diff2','F4_diff2','F5_diff2']]=np.abs(F_i-2*F_i1+F_i2)
            
            #individual_formant=individual_formant.dropna()   #drop the first two rows as there are NaN values due to the difference
            
            folder_formant=folder_formant.append(individual_formant,ignore_index=True)
        
        #folder_formant['disp(F5-F1)']=folder_formant['F5']-folder_formant['F1']
        #folder_formant['disp(F4-F3)']=folder_formant['F4']-folder_formant['F3']
        #folder_formant['disp(F5-F3)']=folder_formant['F5']-folder_formant['F3']
        #folder_formant['disp(F5-F4)']=folder_formant['F5']-folder_formant['F4']
        
        folder_formant['label']=label_index  #new columns for encoding label
        
        all_formant=all_formant.append(folder_formant,ignore_index=True)     
    return all_formant

def get_train_test():
    all_formants=get_all_formants()
    
    y=all_formants['label'].values
 
    only_formants=all_formants.drop(labels=['time','label'],axis=1)
    
    standard_formants=(only_formants-only_formants.mean())/only_formants.std()
    
    X=standard_formants.values   

    assert X.shape[0] == len(y)
    return X,y

In [17]:
word_formant_features=get_all_formants()
#word_formant_features.to_excel("./New_without_CNN/word_all_formant_features.xlsx")

Reading Formants of label -'Formant1':   0%|          | 0/40 [00:00<?, ?it/s]/tmp/ipykernel_22629/3993835764.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  folder_formant=folder_formant.append(individual_formant,ignore_index=True)
Reading Formants of label -'Formant1':   2%|▎         | 1/40 [00:00<00:06,  5.76it/s]/tmp/ipykernel_22629/3993835764.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  folder_formant=folder_formant.append(individual_formant,ignore_index=True)
/tmp/ipykernel_22629/3993835764.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  folder_formant=folder_formant.append(individual_formant,ignore_index=True)
Reading Formants of label -'Formant1':   8%|▊         | 3/40 [00:00<00:03, 10.51it/s]/tmp/i

In [18]:
word_formant_features=word_formant_features.drop(labels=['F4','F5'],axis=1) #reporting result for F1 to F3 only
word_formant_features

,time,F1,F2,F3,label
0,0.025,0.00,247.87,1699.69,0
1,0.031,0.00,281.54,1867.43,0
2,0.038,0.00,307.27,2010.72,0
3,0.044,0.00,312.13,1980.29,0
4,0.050,0.00,312.40,1804.43,0
...,...,...,...,...,...
19682,0.388,426.76,1907.88,2593.99,6
19683,0.394,427.44,1953.37,2824.18,6
19684,0.400,420.76,1962.80,2635.39,6
19685,0.406,380.70,1958.74,2398.14,6


In [19]:
X,y=get_train_test()

Reading Formants of label -'Formant1':   0%|          | 0/40 [00:00<?, ?it/s]/tmp/ipykernel_22629/3993835764.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  folder_formant=folder_formant.append(individual_formant,ignore_index=True)
/tmp/ipykernel_22629/3993835764.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  folder_formant=folder_formant.append(individual_formant,ignore_index=True)
Reading Formants of label -'Formant1':   5%|▌         | 2/40 [00:00<00:02, 13.85it/s]/tmp/ipykernel_22629/3993835764.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  folder_formant=folder_formant.append(individual_formant,ignore_index=True)
/tmp/ipykernel_22629/3993835764.py:31: FutureWarning: The frame.append method is deprecated

In [20]:
# checking batch size

kfold=StratifiedKFold(n_splits=4,shuffle=True,random_state=seed)

for train,test in kfold.split(X,y):    
    print('batch size = '+str(X[train].shape[0]))

batch size = 14765
batch size = 14765
batch size = 14765
batch size = 14766


In [ ]:
kfold=StratifiedKFold(n_splits=4,shuffle=True,random_state=seed)
cv_acc=[]
cv_AUC=[]
cv_f1=[]
cv_kappa=[]

for train,test in kfold.split(X,y):
    y_hot=to_categorical(y)
    
    model_w=Sequential()
    model_w.add(Dense(128, activation='tanh',input_shape=(X[train].shape[1],)))
    model_w.add(Dense(128, activation='tanh'))
    model_w.add(Dense(64, activation='tanh'))
    model_w.add(Dense(32, activation='tanh'))
    model_w.add(Dense(16, activation='tanh'))
    model_w.add(Dense(7, activation='softmax'))
    
    model_w.compile(
        optimizer=opt.Adam(learning_rate=0.005),
        loss='categorical_crossentropy',
        metrics=[['accuracy','AUC']]
    )
    model_w.fit(X[train], y_hot[train], epochs=300, batch_size=X[train].shape[0],verbose=0)
    
    scores=model_w.evaluate(X[test],y_hot[test],verbose=0)
    
    print("%s: %.2f" % (model_w.metrics_names[1],scores[1]))
    print("%s: %.2f" % (model_w.metrics_names[2],scores[2]))
    
    #yhat_classes = model_w.predict_classes(X[test], verbose=0)#deprecated
    yhat_classes=np.argmax(model_w.predict(X[test]), axis=-1)
    
    f1 = f1_score(y[test], yhat_classes,average='weighted')
    print('F1: %.2f' % f1)
    
    kappa = cohen_kappa_score(y[test], yhat_classes)
    print('Cohens kappa: %.2f' % kappa)
    
    cv_acc.append(scores[1])
    cv_AUC.append(scores[2])
    cv_f1.append(f1)
    cv_kappa.append(kappa)
    
    print(' ')
    
print("Overall acc: %.2f (+/- %.2f)" % (np.mean(cv_acc),np.std(cv_acc)))
print("Overall AUC: %.2f (+/- %.2f)" % (np.mean(cv_AUC),np.std(cv_AUC)))
print("Overall f1: %.2f (+/- %.2f)" % (np.mean(cv_f1),np.std(cv_f1)))
print("Overall kappa: %.2f (+/- %.2f)" % (np.mean(cv_kappa),np.std(cv_kappa)))

accuracy: 0.64
auc: 0.91
F1: 0.64
Cohens kappa: 0.58
 
accuracy: 0.63
auc: 0.91
F1: 0.63
Cohens kappa: 0.56
 
accuracy: 0.64
auc: 0.91
F1: 0.64
Cohens kappa: 0.58
 
accuracy: 0.64
auc: 0.91
F1: 0.64
Cohens kappa: 0.58
 
Overall acc: 0.64 (+/- 0.01)
Overall AUC: 0.91 (+/- 0.00)
Overall f1: 0.64 (+/- 0.01)
Overall kappa: 0.58 (+/- 0.01)


In [ ]:
cv_data_df=pd.DataFrame(data={'acc':cv_acc,
                              'AUC':cv_AUC,
                              'F1':cv_f1,
                              'Kappa':cv_kappa}
                       )
cv_data_df.to_csv('/home/rakibul/WORK/RESEARCH/Speech_Recognition/MAIN STUDY/GitHub Repo/Result/New_without_CNN/Word_cv_data_df_'+save_specifier+'.csv')